Improving Computer Vision Accuracy using Convolutions


In [1]:
import tensorflow as tf
from tensorflow.keras import layers
mnist =tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
train_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
    layers.Flatten(),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs = 5)
test_loss = model.evaluate(test_images, test_labels)


Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 3.2662 - accuracy: 0.6640

Your accuracy is probably about 89% on training and 87% on validation...not bad...But how do you make that even better? One way is to use something called Convolutions. I'm not going to details on Convolutions here, but the ultimate concept is that they narrow down the content of the image to focus on specific, distinct, details.

In [2]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images.reshape(len(training_images), 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(len(test_images), 28, 28, 1)
test_images = test_images / 255.0
model = tf.keras.models.Sequential([
    layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=5)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [3]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss, test_acc)


313/313 [==============================] - 3s 9ms/step - loss: 0.2415 - accuracy: 0.9142

It's likely gone up to about 93% on the training data and 91% on the validation data.
Try running it for more epochs -- say about 20, and explore the results! But while the results might seem really good, the validation results may actually go down, due to something called 'overfitting' which will be discussed later.

epochs 수를 늘리면 물론 training set에 대한 결과는 높아진다. 하지만 validation에 대한 accuracy는 떨어진다.
overfitting이 일어났다는 의미로 아래와 같이 초기 conv2D filter 수를 줄여주고 점점 layer가 깊어질 수록 filter수가 증가하는 형식으로 구성을 해서 해결한다.
각 Layer에서의 연산시간/량을 비교적 일정하게 유지하며 시스템의 균형을 맞추는 것이 좋다.
보통 Pooling Layer를 거치면 1/4로 출력이 줄어들기 때문에 Convolution Layer의 결과인 Feature Map의 개수를 4배정도 증가시키면 된다.


In [4]:
class mycallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

In [5]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images.reshape(len(training_images), 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(len(test_images), 28, 28, 1)
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D(2,2),
    # layers.Conv2D(64, (3,3), activation='relu'),
    # layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=20)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1875/1875 [==============================] - 32s 17ms/step - loss: 0.3818 - accuracy: 0.8649

In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss, test_acc)



313/313 [==============================] - 1s 5ms/step - loss: 0.4928 - accuracy: 0.9162